In [5]:
## Init Script
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

# Store student heights in an array
x = np.array([168, 161, 167, 179, 184, 166, 198, 187, 191, 179])

def parametric_bootstrap(dist_type="normal", n_bootstrap=1000, sample_size=None, params=None, stat_func=np.mean, plot=False):
    results = []

    for _ in range(n_bootstrap):
        if dist_type == "normal":
            sample = np.random.normal(loc=params[0], scale=params[1], size=sample_size)
        elif dist_type == "t":
            sample = stats.t.rvs(df=params[0], size=sample_size)
        elif dist_type == "binomial":
            sample = np.random.binomial(n=params[0], p=params[1], size=sample_size)
        elif dist_type == "poisson":
            sample = np.random.poisson(lam=params[0], size=sample_size)
        elif dist_type == "exponential":
            sample = np.random.exponential(scale=1/params[0], size=sample_size)
        elif dist_type == "chi2":
            sample = np.random.chisquare(df=params[0], size=sample_size)
        else:
            raise ValueError(f"Unsupported distribution type: {dist_type}")

        result = stat_func(sample)
        results.append(result)

    results = np.array(results)

    if plot:
        plt.hist(results, bins=30, edgecolor='black', alpha=0.75)
        plt.title(f"Parametric Bootstrap ({dist_type.capitalize()} Distribution)")
        plt.xlabel("Statistic Value")
        plt.ylabel("Frequency")
        plt.show()

    return results


def nonparametric_bootstrap(data, dist_type="normal", n_bootstrap=1000, sample_size=None, plot=False):
    data = np.asarray(data)
    if sample_size is None:
        sample_size = len(data)

    bootstrap_means = []

    for _ in range(n_bootstrap):
        resample = data

        # Parameter estimation
        mu = np.mean(resample)
        sigma = np.std(resample, ddof=1)
        df = len(resample) - 1
        lam = 1 / mu if mu != 0 else 1
        p = mu / np.max(data) if np.max(data) != 0 else 0.5

        # Sampling based on distribution
        if dist_type == "normal":
            sample = np.random.normal(loc=mu, scale=sigma, size=sample_size)
        elif dist_type == "t":
            sample = stats.t.rvs(df=df, size=sample_size)
        elif dist_type == "binomial":
            n = 10
            sample = np.random.binomial(n=n, p=p, size=sample_size)
        elif dist_type == "poisson":
            sample = np.random.poisson(lam=mu, size=sample_size)
        elif dist_type == "exponential":
            sample = np.random.exponential(scale=1/lam, size=sample_size)
        elif dist_type == "chi2":
            sample = np.random.chisquare(df=mu, size=sample_size)
        else:
            raise ValueError("Unsupported distribution type.")

        bootstrap_means.append(np.mean(sample))

    bootstrap_means = np.array(bootstrap_means)

    if plot:
        plt.hist(bootstrap_means, bins=30, edgecolor='black', alpha=0.7)
        plt.title(f"Bootstrap Sample Means ({dist_type.capitalize()} Distribution)")
        plt.xlabel("Mean")
        plt.ylabel("Frequency")
        plt.show()

    return bootstrap_means

## Confidence Interval (CI)
$$
CI=\bar x\pm z\frac{s}{\sqrt n}
$$
- $\bar x$: sample mean
- $z$: confidence level value
- $s$: sample standard deviation
- $n$: sample size

In [ ]:
# Confidence interval for standard deviation and mean
n = 80
mean = 35/80
std = np.sqrt(35/80*(1-23/80))
df = n - 1
alpha = 0.05

conf_int_mean = stats.norm.interval(1-alpha, loc=mean, scale=std/np.sqrt(n))
print(f"Mean CI: {conf_int_mean[0]:.4f}, {conf_int_mean[1]:.4f}     width: {conf_int_mean[1]-conf_int_mean[0]:.4f}")

chi2_lower = stats.chi2.ppf(1 - alpha/2, df)
chi2_upper = stats.chi2.ppf(alpha/2, df)
lower = np.sqrt((df * std**2) / chi2_lower)
upper = np.sqrt((df * std**2) / chi2_upper)
print(f"Std CI:  {lower:.4f}, {upper:.4f}     width: {upper-lower:.4f}")

In [ ]:
vals = [8500, 10300, 6800, 10600, 4900, 6200, 10800, 5700, 5100, 9000]
alpha = 0.01

n = len(vals)
mean = np.mean(vals)
std = stats.sem(vals)
df = n - 1

conf_int_mean = stats.norm.interval(1-alpha, loc=mean, scale=std)
print(f"Mean CI: {conf_int_mean[0]:.2f}, {conf_int_mean[1]:.2f}     width: {conf_int_mean[1]-conf_int_mean[0]:.4f}")

chi2_lower = stats.chi2.ppf(1 - alpha/2, df)
chi2_upper = stats.chi2.ppf(alpha/2, df)
lower = np.sqrt((df * std**2) / chi2_lower)
upper = np.sqrt((df * std**2) / chi2_upper)
print(f"Std CI:  {lower:.2f}, {upper:.2f}     width: {upper-lower:.4f}")

In [ ]:
# Critical value from degrees of freedom
alpha = 0.01
df = 53.627
tails = 2 # 1 for single-tail test and 2 for 2-tailed test

critical_value = stats.t.ppf(1 - alpha / tails, df)
print(f"Critical values: ±{critical_value:.4f}")

## Parametric Bootstrapping

Uses summary statistics of a sample (mean, standard deviation, and number of samples) to simulate a separate sample of similar qualities.

A resampling method that assumes the data follow a specific parametric distribution (e.g., normal, exponential).

**Steps**:
- Fit a theoretical distribution to the original data.
- Generate many random samples from the fitted distribution.
- Compute the statistic of interest from each simulated dataset.
- Use the distribution of these statistics for inference (e.g., confidence intervals).

**Key Points**:
- Assumes a known distribution.
- More efficient if the assumption is correct.
- Can lead to misleading results if the distributional assumption is wrong.

## Non-Parametric Bootstrapping

A distribution-free resampling method that makes no assumptions about the population's distribution.

**Steps**:
- Sample with replacement from the observed data to create many bootstrap samples.
- Calculate the statistic of interest for each sample.
- Use the empirical distribution of the statistic for inference.

**Key Points**:
- Does not assume any distribution.
- More robust to model misspecification.
- May require more computational effort.

## Bootstrapper

Parametric bootstrap simulator for various distributions.

Parameters:
- data (array-like): Observed data to estimate distribution parameters.
- dist_type: 'normal', 't', 'binomial', 'poisson', 'exponential', 'chi2'.
- n_bootstrap: Number of bootstrap iterations.
- sample_size: Size of each simulated sample. Defaults to len(data).
- dist_params (dict): mu, sigma, df, lam, p, n_binom.
- stat_func (callable): Function to compute statistic on simulated samples.
- plot (bool): Whether to plot histogram of bootstrap statistics.

Returns:
- np.ndarray: Array of bootstrap statistics.

Required parameters:
- **normal**: mean, standard deviation
- **t**: degrees of freedom
- **binomial**: attempts per value, success probability
- **poisson**: expected times within interval
- **exponential**: rate
- **chi2**: mu

In [ ]:
# Example usage
# Parametric bootstrap providing parameters
para = parametric_bootstrap(
    dist_type="normal",
    params=[100, 0.8],
    n_bootstrap=10000,
    plot=True
)

# Nonparametric bootstrap providing source sample
nonpara = nonparametric_bootstrap(
    dist_type="normal",
    data=np.random.normal(loc=5.0, scale=1.5, size=100),
    n_bootstrap=500,
    plot=True
)

print(f"Bootstrap estimate of mean: {np.mean(para):.4f}")
print(f"95% CI: {np.percentile(para, 2.5):.4f} to {np.percentile(para, 97.5):.4f}")
